In [ ]:
# 2022-08
# maximize worst-case profits from Neopets Food Club
# (fake-money gambling on Neopets eating contests)

# interesting constraints: <=10 bets on joint outcomes,
# with a slowly-increasing cap on money wagered on each bet
# "odds" seem to be inverse probabilities, not true odds
# i.e. 2:1 "odds" is 50% implied probability

# note that this is very risk-averse: it maximizes
# worst-case outcome, which is more cautious than just
# maximizing EV for worst-case probability distribution
# (because of the mix of single and double bets)
# this is mostly bc it was easier to code this version

# the alpha-maximizing strategy is approximately
# to bet everything on the most likely outcome
# because the bookie gives even odds on the favorite
# (and otherwise rounds odds off in the house's favor)
# this still generally has a >25% chance of winning
# so it's well below Kelly threshold (given my neopoint bankroll)

In [94]:
# wager amounts and profits for mutually-exclusive outcomes

def getbets(odds, mult=1):
    probs = [1/x for x in odds]
    bets = [mult * x/max(probs) for x in probs]
    return bets

def profit(odds):
    bets = getbets(odds)
    cost = sum(bets)
    win = odds[0] * bets[0]
    return win - cost

In [54]:
# convert 2 events w/ 4 outcomes each to 10 mutually-exclusive bets
# by betting on 2 single outcomes from one event and all other pairs

def profit_10(odds1, odds2, solo_i, mult=1):
    # do 2 solo in odds1, plus 8 combinations of the rest
    
    solo = [odds1[i] for i in solo_i]
    notsolo = [odds1[i] for i in range(len(odds1)) if i not in solo_i]
    odds = [x * y for x in notsolo for y in odds2] + [x for x in solo]
    return getbets(odds, mult), profit(odds)

def max_profit_10_dir(odds1, odds2, mult=1):
    # find best two to do solo for odds1
    
    maxprof = -1000
    best = None
    for i in range(len(odds1)):
        for j in range(i):
            bets, prof = profit_10(odds1, odds2, [j, i], mult)
            if prof > maxprof:
                maxprof = prof
                best = [j, i], bets
                
    return maxprof, best

def max_profit_10(odds1, odds2, mult=1):
    # try both ways
    
    maxprof1, best1 = max_profit_10_dir(odds1, odds2, mult)
    maxprof2, best2 = max_profit_10_dir(odds2, odds1, mult)
    
    if maxprof1 > maxprof2:
        return 0, maxprof1, best1
    else:
        return 1, maxprof2, best2

In [95]:
allodds = [
    [13, 13, 2, 13],
    [13, 13, 7, 2],
    [3, 5, 3, 4],
    [9, 9, 13, 2],
    [5, 3, 11, 2]]
[profit(x) for x in allodds]

[0.5384615384615383,
 0.4065934065934065,
 -0.3500000000000001,
 0.4017094017094016,
 -0.24848484848484853]

In [101]:
[(j, i, max_profit_10(allodds[i], allodds[j], LIM)[1]) for i in range(5) for j in range(i)]

[(0, 1, 1.5460693153000848),
 (0, 2, 1.2115384615384617),
 (1, 2, 0.8159340659340666),
 (0, 3, 1.5404339250493102),
 (1, 3, 1.3296703296703303),
 (2, 3, 0.8012820512820511),
 (0, 4, 0.7902097902097909),
 (1, 4, 0.5264735264735272),
 (2, 4, -0.3163636363636364),
 (3, 4, 0.5167055167055166)]

In [103]:
LIM = 6068
d, maxprof, (solo_i, bets) = max_profit_10(allodds[0], allodds[1], LIM)
allodds[0], allodds[1]

([13, 13, 2, 13], [13, 13, 7, 2])

In [93]:
print(maxprof * LIM)
odds1, odds2 = oddses[d], oddses[1-d]

solo = [odds1[i] for i in solo_i]
notsolo = [odds1[i] for i in range(len(odds1)) 
           if i not in solo_i]
labs = [(x, y) for x in notsolo 
        for y in odds2] + [(x,) for x in solo]
print(d, solo_i)
print(sum([int(x+.5) for x in bets]))
[*zip(labs, [int(x+.5) for x in bets])]

9381.548605240914
0 [0, 1]
14893


[((2, 13), 934),
 ((2, 13), 934),
 ((2, 7), 1734),
 ((2, 2), 6068),
 ((13, 13), 144),
 ((13, 13), 144),
 ((13, 7), 267),
 ((13, 2), 934),
 ((13,), 1867),
 ((13,), 1867)]

In [ ]:
# solo: Shipwreck: Orvinn, Fairfax
# all: Shipwreck: Gooblah, Stuff
# all: Lagoon: all

In [87]:
24271 - 14893

9378

```
Current Bets
Round	Bet Info	Amount	Odds	Winnings
8859	Shipwreck: Gooblah the Grarrl
Lagoon: Scurvy Dan the Blade
6068 NP	4:1	24272 NP
8859	Shipwreck: Fairfax the Deckhand
1867 NP	13:1	24271 NP
8859	Shipwreck: Orvinn the First Mate
1867 NP	13:1	24271 NP
8859	Shipwreck: Gooblah the Grarrl
Lagoon: Young Sproggie
1734 NP	14:1	24276 NP
8859	Shipwreck: Gooblah the Grarrl
Lagoon: Puffo the Waister
934 NP	26:1	24284 NP
8859	Shipwreck: Gooblah the Grarrl
Lagoon: Captain Crossblades
934 NP	26:1	24284 NP
8859	Shipwreck: Stuff-A-Roo
Lagoon: Scurvy Dan the Blade
934 NP	26:1	24284 NP
8859	Shipwreck: Stuff-A-Roo
Lagoon: Young Sproggie
267 NP	91:1	24297 NP
8859	Shipwreck: Stuff-A-Roo
Lagoon: Captain Crossblades
144 NP	169:1	24336 NP
8859	Shipwreck: Stuff-A-Roo
Lagoon: Puffo the Waister
144 NP	169:1	24336 NP
Total Possible Winnings	242911 NP
```